<a href="https://colab.research.google.com/github/yomnaFathy/Images-Classification-using-Deep-Neural-Network/blob/main/Flower_Image_Classification_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !pip install py7zr

In [3]:
# import py7zr
# zip_file = "/content/drive/My Drive/datasets/flower_data.7z"

# with py7zr.SevenZipFile(zip_file) as zipObj:
#     zipObj.extractall("/content/drive/My Drive/datasets/")

In [4]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models

In [5]:
data_dir = "/content/drive/My Drive/datasets/flower_data/"

In [6]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(), 
                                       transforms.ToTensor(), 
                                       normalize])

valid_transforms = transforms.Compose([transforms.Scale(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      normalize])

test_transforms = transforms.Compose([transforms.Scale(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      normalize])

train_data = datasets.ImageFolder(data_dir + "train", transform=train_transforms)
valid_data = datasets.ImageFolder(data_dir + "valid", transform=test_transforms)
test_data = datasets.ImageFolder(data_dir + "test", transform=test_transforms)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:257: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [7]:
model = models.vgg16(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
for param in model.parameters():
    param.requires_grad=False

classifier = nn.Sequential(nn.Linear(25088, 4096),
                           nn.ReLU(),
                           nn.Dropout(0.5),
                           nn.Linear(4096, 102),
                           nn.LogSoftmax(dim=1))

model.classifier = classifier

In [9]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=102, bias=True)
  (4): LogSoftmax(dim=1)
)

#### Comparing time consumed using GPU and CPU

In [10]:
import time

In [11]:
for device in ['cuda', 'cpu']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(train_loader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = cuda; Time per batch: 0.002 seconds
Device = cpu; Time per batch: 10.187 seconds


In [12]:
# Test Only to print out some info

for device in ['cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    print("Loss Function:\n", criterion)
    print("Optimization Function:\n", optimizer)
    print("New Model Structure:\n", model)

    for ii, (inputs, labels) in enumerate(train_loader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)
        
        print("Batch of Images:\n", inputs)
        print("Batch Shape:\n", inputs.shape)
        print("Batch Labels:\n", labels)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        print("Loss:\n", loss)
        loss.backward()
        optimizer.step()
        break
        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Loss Function:
 NLLLoss()
Optimization Function:
 Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
New Model Structure:
 VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): R

In [13]:
# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# loss function
criterion = nn.NLLLoss()
# train only classifier parameters
optimizer = optim.Adam(model.classifier.parameters())
# add model to gpu
model.to(device);

In [14]:
# Model Training
epochs = 5
steps = 0 
running_loss = 0
print_every = 30
for epoch in range(epochs):
    for inputs, labels in train_loader:
        steps += 1

        inputs, labels = inputs.to(device), labels.to(device)

        # set gradients to 0, so that parameters updates correctly
        optimizer.zero_grad()

        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        # calculates gradients
        loss.backward()
        # update parameters
        optimizer.step()

        running_loss += loss.item()

        if steps % print_every == 0:
            valid_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in valid_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)

                    valid_loss += batch_loss.item()

                    # calculate accuracy
                    ps = torch.exp(logps)
                    equals = ps.max(dim=1)[1] == labels.data
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Validation loss: {valid_loss/len(valid_loader):.3f}.. "
                  f"Validation accuracy: {accuracy/len(valid_loader):.3f}.. ")
            
            running_loss = 0
            model.train()


Epoch 1/5.. Train loss: 3.737.. Validation loss: 1.755.. Validation accuracy: 0.577.. 
Epoch 1/5.. Train loss: 2.207.. Validation loss: 1.041.. Validation accuracy: 0.723.. 
Epoch 1/5.. Train loss: 1.790.. Validation loss: 0.894.. Validation accuracy: 0.755.. 
Epoch 2/5.. Train loss: 1.683.. Validation loss: 0.767.. Validation accuracy: 0.790.. 
Epoch 2/5.. Train loss: 1.444.. Validation loss: 0.708.. Validation accuracy: 0.807.. 
Epoch 2/5.. Train loss: 1.399.. Validation loss: 0.686.. Validation accuracy: 0.809.. 
Epoch 3/5.. Train loss: 1.308.. Validation loss: 0.589.. Validation accuracy: 0.829.. 
Epoch 3/5.. Train loss: 1.263.. Validation loss: 0.590.. Validation accuracy: 0.824.. 
Epoch 3/5.. Train loss: 1.247.. Validation loss: 0.533.. Validation accuracy: 0.851.. 
Epoch 3/5.. Train loss: 1.261.. Validation loss: 0.498.. Validation accuracy: 0.860.. 
Epoch 4/5.. Train loss: 1.109.. Validation loss: 0.493.. Validation accuracy: 0.871.. 
Epoch 4/5.. Train loss: 1.114.. Validation 

In [15]:
# Model Testing
test_loss = 0
accuracy = 0
model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        logps = model.forward(inputs)
        batch_loss = criterion(logps, labels)

        test_loss += batch_loss.item()
        ps = torch.exp(logps)
        equals = ps.max(dim=1)[1] == labels.data
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

print(f"Test loss: {test_loss/len(test_loader):.3f}.. "
      f"Test accuracy: {accuracy/len(test_loader):.3f}.. ")

Test loss: 0.536.. Test accuracy: 0.865.. 
